In [1]:
from pytorch_modules.model import DistilBERTClass
from pytorch_modules.dataset import MultiLabelDataset
import torch
from pytorch_modules import config
import pandas as pd
from transformers import DistilBertTokenizer
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from pytorch_modules.utils import train, save_model

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
df = pd.read_csv(config.DATA_DIR)
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", truncation=True, do_lower_case=True
)
dataset = MultiLabelDataset(
    dataframe=df, tokenizer=tokenizer, max_len=config.MAX_LEN
)
train_set, val_set = random_split(
    dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42)
)


In [3]:
train_set

In [4]:
trial_pass = train_set[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/stanley/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [5]:
ids = trial_pass['ids'].to(device, dtype = torch.long)
mask = trial_pass['mask'].to(device, dtype = torch.long)
token_type_ids = trial_pass['token_type_ids'].to(device, dtype = torch.long)
targets = trial_pass['targets'].to(device, dtype = torch.float)

In [6]:
model_trial = DistilBERTClass().to(device)
outputs = model_trial(ids, mask, token_type_ids)

In [12]:
print(outputs.squeeze(dim=0).shape, targets.shape)
print(outputs, targets)

torch.Size([6]) torch.Size([6])
tensor([[ 0.0270,  0.0123, -0.0685, -0.0351, -0.2996,  0.0467]],
       device='cuda:0', grad_fn=<AddmmBackward0>) tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')


In [11]:
preds = torch.round(torch.sigmoid(outputs))
output = torch.eq(targets, outputs).sum().item()/len(preds)
output

0.0

In [13]:

train_dataloader = DataLoader(
    dataset=train_set,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    num_workers=config.NUM_WORKERS,
)

val_dataloader = DataLoader(
    dataset=val_set,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    num_workers=config.NUM_WORKERS,
)

model = DistilBERTClass()
model.to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=config.LEARNING_RATE)

results = train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=config.EPOCHS,
    device=device,
)

save_model(model=model, target_dir="models/", model_name="test_model.pt")

  0%|          | 0/1 [00:00<?, ?it/s]

/home/stanley/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/stanley/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None 

Training Epoch 0:   0%|          | 0/12766 [00:00<?, ?it/s]

KeyboardInterrupt: 